# building a stock program

- STEP 1
    - monitor 3 or 4 diverse stocks (yfinance? finnhub?)
    - PCAR, TSLA, AMZN, CRUDE OIL
    - read in stock data for these
- STEP 2
    - pass their own data as endog and the others as exog into a walking sarima
- STEP 2.5
    - feature engeneering. What else can I add as exog to increase accuracy? Unemploymeent? Inflation? etc..
    - How should I incorpoporate things other than Close
- STEP 3
    - set up an account with a small amount of money and let this invest it freely
    - maybe in my schwab? maybe on investopedia?


- Thoughts
    - switch to day change
    - switch from forecasting to just classification sell/hold
    - look into stats forecast package

In [154]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import statsmodels.api as sm
import pmdarima as pm
import warnings
from sklearn.metrics import mean_squared_error
import math
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [155]:
test = yf.Ticker('amzn').history(period='7day')
test

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-08-12,142.050003,143.570007,140.119995,143.550003,47582800,0,0


In [156]:
pcar = yf.Ticker('PCAR').history(period='max')[['Close']]#,'High','Low','Volume']]
amzn = yf.Ticker('AMZN').history(period='max')[['Close']]#,'High','Low','Volume']]
tsla = yf.Ticker('TSLA').history(period='max')[['Close']]#,'High','Low','Volume']]
oil = yf.Ticker('CL').history(period='max')[['Close']]#,'High','Low','Volume']]
comp = yf.Ticker('comp').history(period='max')[['Close']]#,'High','Low','Volume']]
aapl = yf.Ticker('aapl').history(period='max')[['Close']]#,'High','Low','Volume']]


In [157]:
stocks = [pcar,amzn,tsla,oil,comp,aapl]
stock_name = ['pcar','amzn','tsla','oil','comp','aapl']


In [158]:
combo = pd.concat(stocks,axis=1)
combo.columns = stock_name
combo = combo.dropna()
combo.index = pd.to_datetime(combo.index)



In [159]:
# look into endog params within model

def walking_sarimax(data,tt,params):
    walking_preds_list=[]
    j_preds=[]
    acts=[]
    lis = [x for x in combo.columns]
    order_ = params['order']
    seasonal_order_ = params['seasonal_order']
    trend_ = params['trend']
    for stock in lis:
        size = int(len(data)*tt)
        train, test = data[0:size],data[size:len(data)]
        test = np.append(test,np.nan)

        train_endog = train[[stock]]
        test_endog = test[[stock]]
        test_endog_ = [x for x in test_endog.values]
        history_endog = [x for x in train_endog.values]

        exs = lis.copy()
        exs.remove(stock)
        train_exog = train[exs]
        test_exog = test[exs]
        test_exog_ = [x for x in test_exog.values]
        history_exog = [x for x in train_exog.values]

        # needs to be yesterdays info from these stocks. Remove first one, add nan to end (nan added above)

        preds = [np.nan]*len(train)
        for t in range(len(test)):
            model = sm.tsa.statespace.SARIMAX(endog = history_endog,exog = history_exog,\
                order=order_,seasonal_order=seasonal_order_,trend=trend_,\
                    time_varying_regressionbool=True,mle_regression=False,initialization='approximate_diffuse' )

            model_fit = model.fit(disp=False)
            output = model_fit.forecast(exog = test_exog_[t])
            history_endog.append(test_endog_[t])
            history_exog.append(test_exog_[t])
            preds.append(output[0])
            
            if math.isnan(test_endog_[t]) == False:
                j_preds.append(output[0])
                acts.append(test_endog_[t])
        dates = np.append(np.array(data.index.date),(data.index.max()+pd.DateOffset(days=1))) # add tomro
        walking_preds = pd.DataFrame(zip(dates,data[stock],preds),columns=['Date',f'{stock}_Actual',f'{stock}_Pred'])
        walking_preds_list.append(walking_preds)
    walking_preds_all = pd.concat(walking_preds_list,axis=1).T.drop_duplicates().T
    return walking_preds_all,j_preds,acts

In [160]:
ks = [1,2,3]
ks[-1] = 99
ks

[1, 2, 99]

In [161]:
# need to keep exog rows how they are, but instead of endog being todays stock price, it needs to be tomorows stock price
# how to do this?
    # shift stock of interest up?
    # row 1 will have row 2, which is its tomorow
    # last row will have na which makes sense because we dont know it yet
    # will mess up date column because date will be date we are making prediciton (yesterday)

In [162]:
# look into endog params within model

def walking_sarimax(data,tt,params):
    walking_preds_list=[]
    j_preds=[]
    acts=[]
    lis = [x for x in combo.columns]
    order_ = params['order']
    seasonal_order_ = params['seasonal_order']
    trend_ = params['trend']
    for stock in lis:
        exs = lis.copy()
        exs.remove(stock)
        size = int(len(data)*tt)

        # shift endog up
        this_data = data.copy()
        this_data[stock] = this_data[stock].shift(-1) # automatically adds the na for tomoorw
        train, test = this_data[0:size],this_data[size:len(this_data)]

        train_endog = train[[stock]]
        test_endog = test[[stock]]
        test_endog_ = [x for x in test_endog.values]
        history_endog = [x for x in train_endog.values]

        exs = lis.copy()
        exs.remove(stock)
        train_exog = train[exs]
        test_exog = test[exs]
        test_exog_ = [x for x in test_exog.values]
        history_exog = [x for x in train_exog.values]

        preds = [np.nan]*len(train)
        for t in range(len(test_endog_)): # go from one because on 0, we dont have yesterday info
            model = sm.tsa.statespace.SARIMAX(endog = history_endog,exog = history_exog,\
                order=order_,seasonal_order=seasonal_order_,trend=trend_,\
                    time_varying_regressionbool=True,mle_regression=False,initialization='approximate_diffuse' )

            model_fit = model.fit(disp=False)
            output = model_fit.forecast(exog = test_exog_[t])
            history_endog.append(test_endog_[t])
            history_exog.append(test_exog_[t])
            preds.append(output[0])
            
            if math.isnan(test_endog_[t]) == False:
                j_preds.append(output[0])
                acts.append(test_endog_[t])
        dates = np.append(np.array(data.index.date),(data.index.max()+pd.DateOffset(days=1))) # add tomro
        walking_preds = pd.DataFrame(zip(dates,this_data[stock],preds),columns=['Date',f'{stock}_TODAY_ACTUAL',f'{stock}_TOMOROW_PRED'])
        walking_preds_list.append(walking_preds)
    walking_preds_all = pd.concat(walking_preds_list,axis=1).T.drop_duplicates().T
    return walking_preds_all,j_preds,acts

In [163]:
def walking_sarima(data,tt,params):
    walking_preds_list=[]
    j_preds=[]
    acts=[]
    for stock in combo.columns:
        X=data[stock].to_numpy()

        size = int(len(X)*tt)
        train, test = X[0:size],X[size:len(X)]
        #add an np.nan onto the end of train to represent the unknown tomorow
        test = np.append(test,np.nan)
                
        history = [x for x in train]

        preds = [np.nan]*len(train)

        order_ = params['order']
        seasonal_order_ = params['seasonal_order']
        trend_ = params['trend']
        for t in range(len(test)):
            model = sm.tsa.statespace.SARIMAX(history,order=order_,seasonal_order=seasonal_order_,trend=trend_)

            model_fit = model.fit(disp=False)
            output = model_fit.forecast()

            history.append(test[t])
            preds.append(output[0])
            
            if math.isnan(test[t]) == False:
                j_preds.append(output[0])
                acts.append(test[t])

        X = np.append(X,np.nan)
        dates = np.append(np.array(data.index.date),(data.index.max()+pd.DateOffset(days=1))) # add tomro
        walking_preds = pd.DataFrame(zip(dates,X,preds),columns=['Date',f'{stock}_Actual',f'{stock}_Pred'])
        walking_preds_list.append(walking_preds)
    walking_preds_all = pd.concat(walking_preds_list,axis=1).T.drop_duplicates().T
    return walking_preds_all,j_preds,acts

auto = pm.auto_arima(combo.spx, X=combo.drop('spx',axis=1).to_numpy(), start_p=2, d=None, start_q=2, max_p=15, max_d=2, max_q=7, start_P=1, D=None, 
start_Q=1, max_P=5, max_D=1, max_Q=2, max_order=5, m=1, seasonal=True, stationary=False, 
information_criterion='aic', alpha=0.05, test='kpss', seasonal_test='ocsb', stepwise=True, 
n_jobs=1, start_params=None, trend=None, method='lbfgs', maxiter=50, offset_test_args=None, 
seasonal_test_args=None, suppress_warnings=True, error_action='trace', trace=False, random=False, 
random_state=None, n_fits=10, return_valid_fits=False, out_of_sample_size=0, scoring='mse', scoring_args=None, 
with_intercept='auto', sarimax_kwargs=None)

In [164]:
params = {'order':[5,0,0],
          'seasonal_order':[0,0,0,5],
          'trend':'ct'}

In [165]:
outX,pX,aX= walking_sarimax(combo.tail(500),0.9,params)
# endogs is bad right now, try changing to change from just price

print(f'sarimax: {mean_squared_error(pX,aX)}') #115

out,p,a = walking_sarima(combo.tail(100),0.75,params)

print(f'sarima: {mean_squared_error(p,a)}') #208




sarimax: 664402.1682183746
sarima: 208.53056817166373


In [166]:
px.line(outX.set_index('Date'))

# find some way to test
- start with X dollars
- if alg predicts price goes up, hold
- if alg predicts price goes down, sell

In [169]:
def decide(output,stock):
    # if next pred higher than current actual, hold, else, sell
    output = output[['Date',f'{stock}_TODAY_ACTUAL',f'{stock}_TOMOROW_PRED']].dropna()
    todo = []
    act = f'{stock}_TODAY_ACTUAL'
    pre = f'{stock}_TOMOROW_PRED'
    for i in range(len(output)-1):
        if output.iloc[i][act] <= output.iloc[i+1][pre]: # if tomorow pred is bigger than todays actual
            todo.append('HOLD')
        else:
            todo.append('SELL')
    todo.append('?')
    output['Todo'] = todo
    return output

In [170]:
compare_list=[]
for stock in stock_name:
    x = 100
    x2 = x
    x3 = x
    x4=x
    plann=[]
    hold=[]
    perfect=[]
    naive = []
    plan = decide(outX,stock)

    for i in range(1,len(plan)-1):
        row = plan.iloc[i]
        next_row = plan.iloc[i+1]
        last_row = plan.iloc[i-1]
        
        change = ((next_row[1]-row[1])/row[1])
        # using system
        if row.Todo =='HOLD':
            x = x + (x*change)
        # just holding
        x2 = x2+(x2*change)
        # if it was perfect
        if change >0:
            x3 = x3+(x3*change)
        # if positive today positive tmo
        if row[1]>last_row[1]:
            x4 = x4 + x4*change
        
        plann.append(x)
        hold.append(x2)
        perfect.append(x3)
        naive.append(x4)
    
    print(f'{stock}:')
    print(f'    -Perfect: {x3}$')
    print(f'    -Plan: {x}$')
    print(f'    -Hold: {x2}$')
    print(f'    -Naive: {x4}$')

    plan_frame = pd.DataFrame(zip(plan.Date,plann,['plan']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
    hold_frame = pd.DataFrame(zip(plan.Date,hold,['hold']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
    perfect_frame = pd.DataFrame(zip(plan.Date,perfect,['perfect']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
    naive_frame = pd.DataFrame(zip(plan.Date,naive,['naive']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])

    compare= pd.concat([plan_frame,hold_frame,perfect_frame,naive_frame])
    compare_list.append(compare)
all_compare = pd.concat(compare_list)



pcar:
    -Perfect: 124.61478424766997$
    -Plan: 107.17469629756599$
    -Hold: 112.64271648624981$
    -Naive: 109.42947155132919$
amzn:
    -Perfect: 167.8441460320481$
    -Plan: 121.52111968887343$
    -Hold: 133.6592188196275$
    -Naive: 125.7906299242063$
tsla:
    -Perfect: 171.49113219391802$
    -Plan: 123.60969758780813$
    -Hold: 128.95457520146263$
    -Naive: 110.22626768042025$
oil:
    -Perfect: 117.15425927800588$
    -Plan: 99.34499137415366$
    -Hold: 102.40834161534865$
    -Naive: 96.89103995278765$
comp:
    -Perfect: 209.61965732331174$
    -Plan: 108.21502845213571$
    -Hold: 127.7173838044724$
    -Naive: 125.52616569707551$
aapl:
    -Perfect: 139.56227433538814$
    -Plan: 117.99735973464193$
    -Hold: 125.3922110772322$
    -Naive: 108.44412659041011$


In [171]:
all_compare['indicator'] = all_compare['type']+' '+all_compare['stock']

In [172]:
px.line(x=all_compare.date,y=all_compare.val,color=all_compare.indicator)